In [1]:
import pandas as pd

In [5]:
# no_of_kfolds = 10

# #load_kfold_dfs,all_dfs
# #training_all_dfs = pd.DataFrame()
# training_kfold_dfs = dict()
# for kfold in range(1, no_of_kfolds+1):
#     df = pd.read_csv('hybrid_recommender/training_data/combined_predictions_' + str(kfold) + '.csv')
#     training_kfold_dfs[kfold] = df
#     #training_all_dfs = training_all_dfs.append(df)
#     print(training_kfold_dfs[kfold].shape)
# #print(training_all_dfs.shape)
# #training_all_dfs.to_csv('hybrid_recommender/training_data/all_combined_predictions.csv', index=False)


# #testing_all_dfs = pd.DataFrame()
# testing_kfold_dfs = dict()
# for kfold in range(1, no_of_kfolds+1):
#     df = pd.read_csv('hybrid_recommender/testing_data/combined_predictions_' + str(kfold) + '.csv')
#     testing_kfold_dfs[kfold] = df
#     #testing_all_dfs = testing_all_dfs.append(df)
#     print(testing_kfold_dfs[kfold].shape)
# #print(testing_all_dfs.shape)
# #testing_all_dfs.to_csv('hybrid_recommender/testing_data/all_combined_predictions.csv', index=False)

no_of_kfolds = 10
kfold_dfs = dict()
for kfold in range(1, no_of_kfolds+1):
    df = pd.read_csv('hybrid_recommender/combined_predictions_' + str(kfold) + '.csv')
    kfold_dfs[kfold] = df
    print(kfold_dfs[kfold].shape, kfold_dfs[kfold]['uid'].nunique())

(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4710, 10) 942
(4705, 10) 941
(4705, 10) 941


In [6]:
from functools import reduce
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from math import sqrt
import numpy as np

features = ['BaselineOnly_SGD_Tuned_est',
            'Knn_UserBased_ZScore_MSD_Tuned_est',
            'Knn_ItemBased_ZScore_MSD_Tuned_est',
            'Knn_UserBased_Baseline_SGD_Tuned_est',
            'Knn_ItemBased_Baseline_SGD_Tuned_est', 
            'SVD_biased_Tuned_est',
            'SVDpp_biased_Tuned_est']
target = 'r_ui'

In [9]:
def get_prediction_metrics(y_true, y_pred):
    metrics = dict()
    metrics['r2_score'] = r2_score(y_true, y_pred)
    metrics['mean_absolute_error'] = mean_absolute_error(y_true, y_pred)
    metrics['rmse'] = sqrt(mean_squared_error(y_true, y_pred))
    return metrics

def get_avg_test_metrics(kfold_test_metrics):
    metrics = dict()
    metrics['r2_score'] = []
    metrics['mean_absolute_error'] = []
    metrics['rmse'] = []
    
    for test_metrics in kfold_test_metrics:
        metrics['r2_score'].append(test_metrics['r2_score'])
        metrics['mean_absolute_error'].append(test_metrics['mean_absolute_error'])
        metrics['rmse'].append(test_metrics['rmse'])
    #print(metrics)
    avg_metrics = dict()
    avg_metrics['r2_score'] = round(np.mean(metrics['r2_score']), 4)
    avg_metrics['mean_absolute_error'] = round(np.mean(metrics['mean_absolute_error']), 4)
    avg_metrics['rmse'] = round(np.mean(metrics['rmse']), 4)
    return avg_metrics

def get_avg_metrics(estimator, kfold_dfs):
    #print(estimator)
    kfold_test_metrics = []
    for kfold in range(1, no_of_kfolds+1):
        #print('*'*40)
        #print('kfold_', str(kfold))
        train_df = pd.DataFrame()
        test_df = pd.DataFrame()
        for kfold_id, df in kfold_dfs.items():
            if kfold == kfold_id:
                test_df = df
            else:
                train_df = train_df.append(df)
        #print("train_df : {} test_df : {}".format(train_df.shape, test_df.shape))
        train_users = set(train_df['uid'].unique())
        test_users = set(test_df['uid'].unique())
        common_users = train_users & test_users
        #print("No of train_users  : ", len(train_users))
        #print("No of test_users   : ", len(test_users))
        #print("No of common_users : ", len(common_users))
        #input()

        train_X, train_y = train_df[features], train_df[target]    
        #print(train_X.shape, train_y.shape)
        test_X, test_y = test_df[features], test_df[target]
        #print(test_X.shape, test_y.shape)

        model = estimator.fit(train_X, train_y)

        #predicted_train_y = model.predict(train_X)
        #train_metrics = get_prediction_metrics(train_y, predicted_train_y)
        #print(train_metrics)
        predicted_test_y = model.predict(test_X)
        test_metrics = get_prediction_metrics(test_y, predicted_test_y)
        #print(test_metrics)
        kfold_test_metrics.append(test_metrics)
    #print(kfold_test_metrics)
    avg_metrics = get_avg_test_metrics(kfold_test_metrics)
    #print('r2 : {:05.4f}, mae : {:05.4f}, rmse : {:05.4f}'.format(avg_metrics['r2_score'], 
    #                                                 avg_metrics['mean_absolute_error'],
    #                                                 avg_metrics['rmse']))
    #print(avg_metrics)
    #print('*'*40)
    return avg_metrics

# def get_train_test_metrics(estimator, training_df, testing_df):
#     results = dict()
#     results['train'] = get_avg_metrics(estimator, training_df)
#     results['test'] = get_avg_metrics(estimator, testing_df)
#     return results

In [13]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

#Knn_UserBased_Baseline_SGD_Tuned                                      0.6956
#                                                                      0.6946

default_models = [
    LinearRegression(),
    Ridge(),
    Lasso(),
    ElasticNet(),
    SGDRegressor(max_iter=1000, tol=1e-3),
    RandomForestRegressor(),
    GradientBoostingRegressor()
]
for model in default_models:
    #results = get_train_test_metrics(model, training_kfold_dfs, testing_kfold_dfs)    
    #print("{:40s} Train RMSE : {}, Test RMSE : {}".format(type(model).__name__, 
    #                                                      results['train']['rmse'],
    #                                                      results['test']['rmse']))
    results = get_avg_metrics(model, kfold_dfs)
    print("{:40s} RMSE : {}".format(type(model).__name__, results['rmse']))

LinearRegression                         RMSE : 0.6918
Ridge                                    RMSE : 0.6918
Lasso                                    RMSE : 0.8219
ElasticNet                               RMSE : 0.8219
SGDRegressor                             RMSE : 0.6983
RandomForestRegressor                    RMSE : 0.7363
GradientBoostingRegressor                RMSE : 0.6898


In [14]:
tuned_models = [
    LinearRegression(),
    Ridge(alpha=10),
    Lasso(alpha=0.0001),
    ElasticNet(alpha=0.01, l1_ratio=0.2),
    SGDRegressor(alpha=0.0001, l1_ratio=0.1, loss='squared_loss', penalty='elasticnet', tol=0.0001),
    #RandomForestRegressor(max_depth=15, n_estimators=150),
    #GradientBoostingRegressor(learning_rate=0.1, max_depth=7, n_estimators=100)
]
for model in tuned_models:
    #results = get_train_test_metrics(model, training_kfold_dfs, testing_kfold_dfs)
    #print("{:40s} Train RMSE : {}, Test RMSE : {}".format(type(model).__name__, 
    #                                                      results['train']['rmse'],
    #                                                      results['test']['rmse']))
    results = get_avg_metrics(model, kfold_dfs)
    print("{:40s} RMSE : {}".format(type(model).__name__, results['rmse']))

LinearRegression                         RMSE : 0.6918
Ridge                                    RMSE : 0.6918
Lasso                                    RMSE : 0.6918
ElasticNet                               RMSE : 0.6921
SGDRegressor                             RMSE : 0.696


In [9]:
from sklearn.svm import SVR

model = SVR()

results = get_train_test_metrics(SVR(), training_kfold_dfs, testing_kfold_dfs)
print("{:40s} Train RMSE : {}, Test RMSE : {}".format(type(model).__name__, 
                                                      results['train']['rmse'],
                                                      results['test']['rmse']))

GradientBoostingRegressor                Train RMSE : 0.7373, Test RMSE : 0.7365
